# G2Engine Guide - Why Entities

More information:

1. [GitHub repository](https://github.com/Senzing/docker-jupyter)
1. [Senzing documentation](http://docs.senzing.com/?python#g2config)
1. [G2Engine Reference](senzing-G2Engine-reference.ipynb)

## Table of contents

1. [Prepare environment](#Prepare-environment)
    1. [Initialize Senzing configuration](#Initialize-Senzing-configuration)
    1. [Initialize python environment](#Initialize-python-environment)
    1. [Helper class for JSON rendering](#Helper-class-for-JSON-rendering)
    1. [Utility functions](#Utility-functions)
    1. [System path](#System-path)
    1. [Initialize variables](#Initialize-variables)
1. [G2Engine](#G2Engine)
    1. [G2Engine initialization](#G2Engine-initialization)
    1. [Purge prior repository](#Purge-prior-repository)
    1. [Insert records](#Insert-records)
    1. [Ask Why](#Ask-Why)
    1. [Start constructing final report](#Start-constructing-final-report)
    1. [Get the main document data sections](#Get-the-main-document-data-sections)
    1. [Get the basic why results](#Get-the-basic-why-results)
    1. [Get the matching candidate keys for the record](#Get-the-matching-candidate-keys-for-the-record)
    1. [Determine the best matches for each feature type](#Determine-the-best-matches-for-each-feature-type)
    1. [Print the general Why results](#Print-the-general-Why-results)
    1. [Print the matching criteria](#Print-the-matching-criteria)
    1. [Print the candidate keys](#Print-the-candidate-keys)
    1. [Shut down the engine](#Shut-down-the-engine)

## Prepare environment 

### Initialize Senzing configuration

Run [senzing-G2ConfigMgr-reference.ipynb](senzing-G2ConfigMgr-reference.ipynb)
to install a Senzing Engine configuration in the database.

### Initialize python environment

In [ ]:
import argparse
import json
import operator
import os
import sys

# For RenderJSON

import uuid
from IPython.display import display_javascript, display_html, display

### Helper class for JSON rendering

A class for pretty-printing JSON.
Not required by Senzing, 
but helps visualize JSON.

In [ ]:
class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        elif isinstance(json_data, bytearray):
            self.json_str = json_data.decode()
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height:100%; width:100%; background-color: LightCyan"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

### Utility functions

In [ ]:
def calculate_score_bucket_level(bucketString = None):
    score_bucket = {
        'SAME': 0,
        'CLOSE': 1,
        'LIKELY': 2,
        'PLAUSIBLE': 3, 
        'UNLIKELY': 4,
        'NO_CHANCE': 5,
    }
    return score_bucket.get(bucketString, 6)

### System path

Update system path.

In [ ]:
python_path = "{0}/python".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))
sys.path.append(python_path)

### Initialize variables

Create variables used for G2Engine.

In [ ]:
module_name = 'pyG2EngineForWhyEntity'

config_path = os.environ.get("SENZING_ETC_DIR", "/etc/opt/senzing")
support_path = os.environ.get("SENZING_DATA_VERSION_DIR", "/opt/senzing/data")

resource_path = "{0}/resources".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))

sql_connection = os.environ.get(
    "SENZING_SQL_CONNECTION", "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db")

verbose_logging = False
datasource_code = "TEST"
load_id = None

senzing_config_dictionary = {
    "PIPELINE": {
        "CONFIGPATH": config_path,        
        "SUPPORTPATH": support_path,
        "RESOURCEPATH": resource_path
    },
    "SQL": {
        "CONNECTION": sql_connection,
    }
}

senzing_config_json = json.dumps(senzing_config_dictionary)

## G2Engine

In [ ]:
from G2Engine import G2Engine

### G2Engine initialization

Details at [G2Engine initialization](senzing-G2Engine-reference.ipynb#G2Engine-initialization).

In [ ]:
g2_engine = G2Engine()

return_code = g2_engine.initV2(
    module_name,
    senzing_config_json,
    verbose_logging)

print("Return Code: {0}".format(return_code))

### Purge prior repository

Details at [G2Engine.purgeRepository](senzing-G2Engine-reference.ipynb#purgeRepository).

In [ ]:
g2_engine.purgeRepository()

### Insert records

Details at [G2Engine.addRecord](senzing-G2Engine-reference.ipynb#addRecord).

#### Insert record 1

In [ ]:
record_id = "1"
data = {
    "NAMES": [
        {
            "NAME_TYPE": "PRIMARY",
            "NAME_LAST": "Bob",
            "NAME_FIRST": "Max"
        }
    ],
    "PHONE_NUMBER": "123-456-7899"
}
data_as_json = json.dumps(data)

return_code = g2_engine.addRecord(
    datasource_code,
    record_id,
    data_as_json,
    load_id)

print("Return Code: {0}".format(return_code))

#### Insert record 2

In [ ]:
record_id = "2"
data = {
    "NAMES": [
        {
            "NAME_TYPE": "PRIMARY",
            "NAME_LAST": "Robart",
            "NAME_FIRST": "Max",
            "MIDDLE_NAME": "S"
        }
    ],
    "ADDR_FULL": "456 Funny ST"
}
data_as_json = json.dumps(data)

return_code = g2_engine.addRecord(
    datasource_code,
    record_id,
    data_as_json,
    load_id)

print("Return Code: {0}".format(return_code))

#### Insert record 3

In [ ]:
record_id = "3"
data = {
    "NAMES": [
        {
            "NAME_TYPE": "PRIMARY",
            "NAME_LAST": "Robert",
            "NAME_FIRST": "Max",
            "MIDDLE_NAME":"Stewart"
        }
    ],
    "PHONE_NUMBER": "123-456-7899",
    "ADDR_FULL": "456 Funny st"
}
data_as_json = json.dumps(data)

return_code = g2_engine.addRecord(
    datasource_code,
    record_id,
    data_as_json,
    load_id)

print("Return Code: {0}".format(return_code))

### Ask Why

Details at [G2Engine.whyEntityByRecordID](senzing-G2Engine-reference.ipynb#whyEntityByRecordID).

In [ ]:
record_id = "2"
response_bytearray = bytearray()

return_code = g2_engine.whyEntityByRecordID(
    datasource_code,
    record_id,
    response_bytearray)

why_entity_dictionary = json.loads(response_bytearray.decode())

print("Return Code: {0}".format(return_code))
RenderJSON(response_bytearray)

### Start constructing final report

In [ ]:
final_result = {
    "Data_source": datasource_code,
    "Record_ID": record_id
}

### Get the main document data sections

In [ ]:
for why_result in why_entity_dictionary.get("WHY_RESULTS", []):
    for focus_record in why_result.get("FOCUS_RECORDS", []):
        if focus_record.get("DATA_SOURCE") == final_result.get("Data_source") \
        and focus_record.get("RECORD_ID") == final_result.get("Record_ID"):
            break
     
final_result["Entity_ID"] = why_result['ENTITY_ID']
RenderJSON(why_result)

### Get the basic why results

In [ ]:
match_info = why_result.get("MATCH_INFO", {})
final_result["Why_key"] = match_info.get("WHY_KEY")
final_result["Why_ER_rule_code"] = match_info.get("WHY_ERRULE_CODE")

### Get the matching candidate keys for the record

In [ ]:
name_feature_dictionary = {}

for candidate_key, candidate_values in match_info.get("CANDIDATE_KEYS", []).items():
    if candidate_key not in name_feature_dictionary:
        name_feature_dictionary[candidate_key] = []    
    for candidate_value in candidate_values:
        name_feature_dictionary[candidate_key].append(candidate_value.get("FEAT_DESC")) 
        
RenderJSON(name_feature_dictionary)        

### Determine the best matches for each feature type

In [ ]:
best_behavior_level_by_types = {}
best_feature_match_by_types = {}

for feature_score_key, feature_score_values in match_info.get("FEATURE_SCORES", {}).items():
    
    if feature_score_key not in best_feature_match_by_types:
        best_behavior_level_by_types[feature_score_key] = calculate_score_bucket_level()
        
    for feature_score_value in feature_score_values:
        score_bucket = feature_score_value.get("SCORE_BUCKET")
        score_bucket_level = calculate_score_bucket_level(score_bucket)
        
        if best_behavior_level_by_types[feature_score_key] >= score_bucket_level:
            best_behavior_level_by_types[feature_score_key] = score_bucket_level
            best_feature_match_by_types[feature_score_key] = {
                "SCORE_BUCKET": score_bucket,
                "FEAT": feature_score_value.get("INBOUND_FEAT")
            }

RenderJSON(best_feature_match_by_types)

### Print the general "Why" results

In [ ]:
RenderJSON(final_result)        

### Print the matching criteria

In [ ]:
print('Compared Features:')

for key, value in sorted(best_feature_match_by_types.items()):
    print("    {0}: '{1}' ({2})".format(key, value.get("FEAT"), value.get("SCORE_BUCKET")))

### Print the candidate keys

In [ ]:
print('Matching record candidate keys:')

for key, values in sorted(name_feature_dictionary.items()):
    print("    {0}:".format(key))
    values.sort()
    for value in values:
        print("        '{0}'".format(value))

### Shut down the engine

Details at [G2Engine.destroy](senzing-G2Engine-reference.ipynb#destroy).

In [ ]:
return_code = g2_engine.destroy()

print("Return Code: {0}".format(return_code))